# Assessment Problems

***
***

## Problem 1: Extending the Lady Tasting Tea

---

### Research and Development

---

#### Research Context

The [*Lady Tasting Tea*](https://en.wikipedia.org/wiki/Lady_tasting_tea) experiment, introduced by Sir Ronald A. Fisher in 1935, is a classic example of **design of experiments** and **hypothesis testing**.
A lady claimed she could distinguish whether milk was poured into a cup before or after the tea was added.

The original experiment used eight cups: four milk-first ('M') and four tea-first ('T').
The probability of perfectly identifying the milk-first cups by chance was very low (~0.0143).

Extending the experiment to twelve cups, with four milk-first and eight tea-first, allows us to **estimate the likelihood of success by chance in a larger experiment**.

---

#### Development Considerations

* **Representation:** Using 'M' and 'T' provides a clear and simple way to distinguish between milk-first and tea-first cups.  
* **Randomisation:** Each trial is independent, reflecting the null hypothesis.  
* **Simulation scale:** The experiment uses simple loop-based logic to maintain clarity and transparency in the implementation. 
* **Statistical insight:** Demonstrates probability estimation with explicit loops.  

***

### Experiment Setup

---

The process begins with the import of NumPy, which provides the foundation for randomisation and numerical computations throughout the experiment.

In [ ]:
import numpy as np


Before running the experiment, a fixed random seed is set so that NumPy gives the same random choices every time. This makes the results easier to verify later and helps maintain consistency if someone else wants to repeat the same test.

[The Curious Case of 42: Why Python Loves This Number for Randomization](https://medium.com/ai-simplified-in-plain-english/python-random-seed-42-125a3f2e068f) articule on the Medium website gives interesting explain of why the number 42 is often used by programmers.

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)


The experiment proceeds by defining the total number of cups, along with how many are milk-first and tea-first.

In [ ]:
# Total cups and their distribution
total_cups = 12
milk_first = 4
tea_first = 8


Set how many times the experiment will run and start counting successful guesses.

In [ ]:
# Number of trials
n_trials = 1_000_000
successes = 0


Create the initial arrangement of cups before any shuffling takes place.
To understand how to use `numpy.array`, please visit the official documentation: [NumPy Array](https://numpy.org/doc/stable/reference/generated/numpy.array.html).


In [ ]:
# True labels
true_labels = np.array(['M'] * milk_first + ['T'] * tea_first)

# Print initial setup
print("Initial cup setup:", true_labels)
